In [1]:
import tweepy

The following cell lines are just to read your personal Twitter parameters from a file
instead of hardcoding them into your program. A still better approach would be to encrypt them
in the file and decrypt them in the program. Note that readline automatically appends a newline charater, so we strip it off with the .strip('\n') function.

In [2]:
meFilename = 'me.txt'
meFile = open(meFilename,'r') # add a pathname if needed
consumer_key = str(meFile.readline()).strip('\n')
consumer_secret = meFile.readline().strip('\n')
access_token = meFile.readline().strip('\n')
access_token_secret = meFile.readline().strip('\n')
meFile.close()

The result will be something like the following (but with the values from your text file):

consumer_key = 'UIiKZVqFAlBZAyxdIuMK7CMtDz'

consumer_secret = 'loiPCSBlZVvgzrFFJ5ZaA0DOzMeaQ8dvJ4Qqg5i80wUSCHXRKKb'

access_token = '550502130-CAu3LZrUoDcLNG4kAUu4swgKAAGn7Ap9FBSelaAn6'

access_token_secret = 'EP6nX8nP4FSoEj2gu47mGPrB43nVRxn6J8vFWeAUBsHS8S'

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)
maxTweets = 10
tweetNum = 0

public_tweets = api.home_timeline()
for tweet in public_tweets:
    if tweetNum < maxTweets:
        tweetNum +=1 
        print (tweet.text)
    else:
        break

Get into the Halloween spirit with these geeked out pumpkins 🎃 https://t.co/63DZvcD9gR https://t.co/ghM4Agp2FP
I wish the self-defeating stupidity of the UK voting Brexit was as easily reversible as the self-defeating stupidity of the US voting Trump.
Want to transport yourself to another world? These sci-fi authors have got you covered. https://t.co/p5BLgsVu1R
Jeepers creepers! It's a creature double feature! https://t.co/I4jhRVvaRu https://t.co/mXrrd74WsZ
RT @InternetHippo: Terrorist: The code to disarming the nuke is at the end of this 280-character tweet. You need only read it

Me: Folks we…
lol hugely flattered, the WikiTribune team have been given my t-shirts https://t.co/i5d6t1E03S

Get yours here!… https://t.co/qCD57wt8Bm
RT @bechter: Apparently Warheads get their sourness from malic acid, which can also be found in Granny Smith apples. Huh! https://t.co/L3yG…
RT @annieminoff: Just another day at the @scifri office, eating vomit candy w/ flavor historian @thebirdisgone. Glad we

In [4]:
# Get the User object for twitter...
user = api.get_user('twitter')

In [26]:
u = str(user)
print(len(u))
print(u[:900] + '...')
print()
print('...' + u[-900:])

6566
User(_api=<tweepy.api.API object at 0x7fc9b71ccb38>, _json={'id': 783214, 'id_str': '783214', 'name': 'Twitter', 'screen_name': 'Twitter', 'location': 'San Francisco, CA', 'profile_location': None, 'description': 'Your official source for what’s happening.\n\nNeed a hand? Visit https://t.co/jTMg7YsLw5', 'url': 'https://t.co/gN5JJwhQy7', 'entities': {'url': {'urls': [{'url': 'https://t.co/gN5JJwhQy7', 'expanded_url': 'https://blog.twitter.com/', 'display_url': 'blog.twitter.com', 'indices': [0, 23]}]}, 'description': {'urls': [{'url': 'https://t.co/jTMg7YsLw5', 'expanded_url': 'https://support.twitter.com', 'display_url': 'support.twitter.com', 'indices': [63, 86]}]}}, 'protected': False, 'followers_count': 62056962, 'friends_count': 170, 'listed_count': 90700, 'created_at': 'Tue Feb 20 14:35:54 +0000 2007', 'favourites_count': 5197, 'utc_offset': -25200, 'time_zone': 'Pacific Time (US & ...

...nd_color='ACDED6', profile_background_image_url='http://pbs.twimg.com/profile_backgroun

In [11]:
print (user.screen_name)
print (user.followers_count)
    

Twitter
62056962


Sentiment Analysis using tweepy,NLTK and Textblob

Using OAuth to make connection twitter



In [39]:
import tweepy
import time
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import os
import re
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
 
api = tweepy.API(auth)

Collecting tweets in json file

In [52]:
class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open('python.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)

The key attributes of the tweets pulled out are :

    text: the text of the tweet itself

    created_at: the date of creation

    favorite_count, retweet_count: the number of favourites and retweets

favorited, retweeted: boolean stating whether the authenticated user (you) have favourited or retweeted this tweet

lang: acronym for the language (e.g. “en” for english)

id: the tweet identifier

place, coordinates, geo: geo-location information if available

user: the author’s full profile

entities: list of entities like URLs, @-mentions, hashtags and symbols

in_reply_to_user_id: user identifier if the tweet is a reply to a specific user

in_reply_to_status_id: status identifier id the tweet is a reply to a specific status

Processing tweets

Tokenizing the tweet

Tokenizing @-mentions, emoticons, URLs and #hash-tags as individual tokens.

In [53]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

The regular expressions are compiled with the flags re.VERBOSE, to allow spaces in the regexp to be ignored (see the multi-line emoticons regexp), and re.IGNORECASE to catch both upper and lowercases. Thetokenize() function simply catches all the tokens in a string and returns them as a list.


In [54]:
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)

Removing stop-words,punctuations and rt,via words :

In [55]:
import nltk

In [56]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [57]:
from nltk.corpus import stopwords
import string

punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']



In order to keep track of the frequencies while we are processing the tweets, we can use collections.Counter() which internally is a dictionary (term: count) with some useful methods like most_common():

In [51]:
with open('python.json', 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]
        # Update the counter
	    #terms_single = set(terms_all)
		# Count hashtags only
        terms_hash = [term for term in preprocess(tweet['text']) 
              if term.startswith('#')]
# Count terms only (no hashtags, no mentions)
        terms_only = [term for term in preprocess(tweet['text']) 
              if term not in stop and
              not term.startswith(('#', '@'))] 
              # mind the ((double brackets))
              # startswith() takes a tuple (not a list) if 
              # we pass a list of inputs
####Python TextBlob Sentiment Analysis

FileNotFoundError: [Errno 2] No such file or directory: 'python.json'

Sentiment Analysis refers to the process of taking natural language to identify and extract subjective information. You can take text, run it through the TextBlob and the program will spit out if the text is positive, neutral, or negative by analyzing the language used in the text.

Sentiment Analysis	
Text	If that is not cool enough for you than that is a you problem.
Polarity	-0.0875
Subjectivity	0.575
Classification	neg
P_Pos	0.344455873
P_Neg	0.655544127
What does that mean?

Polarity - a measure of the negativity, the neutralness, or the positivity of the text
Classification - either pos or neg indicating if the text is positive or negative
To calculate the overall sentiment, we look at the polarity score:
    Positive – from .01 to 1
    Neutral – 0
    Negative – from –.01 to -1


In [ ]:
for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        blob = TextBlob(tweet["text"])
        cout+=1
        lis.append(blob.sentiment.polarity)
        #print blob.sentiment.subjectivity
        #print (os.listdir(tweet["text"]))
        if blob.sentiment.polarity < 0:
            sentiment = "negative"
            neg+=blob.sentiment.polarity
            n+=1
        elif blob.sentiment.polarity == 0:
            sentiment = "neutral"
            net+=1
        else:
            sentiment = "positive"
            pos+=blob.sentiment.polarity
            p+=1